## Automated Amazon Bedrock Prompt Flow Generation with LLM in Amazon Bedrock 

This notebook shows an example of automatically generating a Amazon Bedrock Prompt Flows (https://aws.amazon.com/bedrock/prompt-flows/) from a description of a generative AI framework.

It is intended to assist the framework/code generation with the Prmopt Flow's visual aid.

The automatically generated the prompt flow deployment code may need a user's review and correction.

We will use Anthropic's Claude Sonnet 3.5 model as backbone for this example.

We will start with a description of a RAG framework with additional modules (e.g., current date, web search, etc.) to generate a prompt flow as shown below.

**Input Description**

```
You are an AI assistant who is well-versed in Amazon Bedrock Knowledge base designed to answer a user's question.
Please generate a code about Amazon Bedrock Knowledge Base only when a user asks for it.
Do NOT use your own knowledge as facts in answers. 
Please output a list in order of the most valuable data sources for a knowledge base.
Don't restate the instructions.
```

**Automatically Generated Prompt Flow**

![alt text](prompt_flow_asset/example_pf_output.png "Automatically Generated Prompt Flow")

Let's start!

### Load packages

* ```pf_design_module```:  Design a Bedrock prompt flow from a user’s description. 
* ```pf_implementation_module```: Based on the design from the design module, implement a code that deploys a Bedrock prompt flow.
* ```pf_user_input_identifier```: (Optional) Identify the configurations that need user's inputs in the code generated by the implementation module
* ```pf_code_update_w_user_input```: (Optional) Update the code with the user's inputs on the configurations.
* ```pf_code_execution```: Simple function to execute the prompt flow deployment code with the user's inputs

The optional modules can be skipped if a user manually types in the required configurations.

In [1]:
from prompt_flow_generation.bedrock_helper import SONNET35_MODEL_ID
from prompt_flow_generation.flow_generation import pf_design_module, pf_implementation_module, pf_user_input_identifier, pf_code_update_w_user_input, pf_code_execution

### Prompt Flows Designer (PFD) Module

PFD module designs the Bedrock prompt flows based on the description of a framework or a query provided by a user. 

The available Bedrock prompt flows nodes information (https://docs.aws.amazon.com/bedrock/latest/userguide/flows-nodes.html) and their JSON formats are provided to a LLM as a context information along with a system prompt.

We selected nodes that are essential for a RAG framework (Flow input, Flow output, Prompt, Knowledge base, Lambda function) for this example.

The PFD module generates the explanation on a prompt flows design along with the JSON configuration of each node. 

The generated prompt flows design is passed to Prompt Flows Implementation (PFI) module for a Python script implementation. 

In [2]:
system_description = """You are an AI assistant who is well-versed in Amazon Bedrock Knowledge base designed to answer a user's question.
Please generate a code about Amazon Bedrock Knowledge Base only when a user asks for it.
Do NOT use your own knowledge as facts in answers. 
Please output a list in order of the most valuable data sources for a knowledge base.
Don't restate the instructions.
"""

pf_design = pf_design_module( system_description=system_description, max_tokens=4000, model_id = SONNET35_MODEL_ID )
print( pf_design )

Based on the given description, I'll design a prompt flow that utilizes a Knowledge Base node to answer user questions about Amazon Bedrock Knowledge Base and a Prompt node to generate code when requested. Here's the flow design:

1. Flow Input Node:
```json
{
   "name": "UserInput",
   "type": "Input",
   "outputs": [
        {
            "name": "document",
            "type": "String"
        }
    ],
    "configuration": {
        "input": "User's question or request"
    }
}
```

2. Knowledge Base Node:
```json
{
    "name": "BedrockKnowledgeBase",
    "type": "KnowledgeBase",
    "inputs": [
       {
            "name": "retrievalQuery",
            "type": "String",
            "expression": "$.data"
        }
    ],
    "outputs": [
        {
            "name": "retrievalResults",
            "type": "Array"
        }
    ],
    "configuration": {
        "knowledgeBase": {
            "knowledgeBaseId": "bedrock-knowledge-base-id"
        }
    }
}
```

3. Prompt Node:
```js

### Prompt Flow Implementation (PFI) Module 

PFI module implements a Python script that deploys the Bedrock prompt flows via boto3. 

It takes the original user’s description/query and the prompt flows designs generated by the PFD module. 

It generates a full Python script that deploys the Bedrock prompt flows based on a user’s description/query and save it to a python file to your designated file path.

**Note**

A user needs to provide actual configurations of the deployment script, e.g., Lambda ARN, Knowledge Base ID, an execution role for the Bedrock prompt flow deployment, etc. 

The user input identifier module ```pf_user_input_identifier``` and code update module  ```pf_code_update_w_user_input``` can be used to assist you to update the code as an optional step as shown in the below steps.


In [3]:
output_py_file_path = "prompt_flow_test/prompt_flow_aws_rag.py"

pf_code = pf_implementation_module( system_description=system_description, 
                                   pf_design=pf_design, 
                                   max_tokens=6000, 
                                   model_id = SONNET35_MODEL_ID, 
                                   save=True, 
                                   output_path=output_py_file_path )

print( pf_code )


import boto3
from datetime import datetime
import json

bedrock_agent = boto3.client(service_name="bedrock-agent", region_name="us-east-1")

# Create the prompt
response = bedrock_agent.create_prompt(
    name=f"AIAssistantPrompt-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="AI Assistant prompt for Amazon Bedrock Knowledge Base",
    variants=[
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 3000,
                    "temperature": 0,
                    "topP": 0.1,
                    "topK": 250,
                }
            },
            "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
            "name": "variant-001",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "user_query"},
                        {"name": "kb_results"}
                    ],
                    "text": """You are an AI assistant who is

### User Input Identifier (UII)

The UII module identifies the required configurations in the implemented code to guide a user about what configurations need to be added.

In [4]:
user_input_guidance = pf_user_input_identifier( pf_code, model_id=SONNET35_MODEL_ID )
print( user_input_guidance )

To complete this code, the user needs to provide the following inputs:

1. AWS Region: The code is currently set to "us-east-1". If the user wants to use a different region, they should update the `region_name` parameter in the `boto3.client()` call.

2. IAM Role ARN: Replace the placeholder ARN in the `flow_role` variable with the actual ARN of the IAM role that has the necessary permissions for Bedrock Agent to execute the flow.

   ```python
   flow_role = "arn:aws:iam::123456789012:role/BedrockFlowRole"  # Replace with your actual role ARN
   ```

3. Knowledge Base ID: Replace the placeholder ID in the "BedrockKnowledgeBase" node configuration with the actual ID of the Amazon Bedrock Knowledge Base.

   ```python
   "knowledgeBaseId": "bedrock-knowledge-base-id"  # Replace with your actual knowledge base ID
   ```

4. AWS Credentials: Ensure that the AWS credentials are properly configured in the environment where this script will run. This can be done through environment variables

### Code Update with User Input

The code update module updates the code generated by the implementation module with the required configurations based on the user's inputs. 

Please review the user guidance provided by the UII module and enter the required configuraitons in natural language.

You will need the AWS account ID, the IAM Role name, and the region that you used to create the Knowledge Base in the previous session.

In [5]:
updated_output_file_path = "prompt_flow_test/prompt_flow_aws_rag_user_input.py"

user_input_msg = """
The Account ID is 613305653031. The IAM role name is AmazonBedrockExecutionRoleForKnowledgeBase_1112039-f. The region is us-east-1.
The knowledge base ID is HHGXZA7BH9.
"""

pf_updated_code = pf_code_update_w_user_input( code_str=pf_code, user_input=user_input_msg, save=True, output_path=updated_output_file_path )
print( pf_updated_code )



import boto3
from datetime import datetime
import json

bedrock_agent = boto3.client(service_name="bedrock-agent", region_name="us-east-1")

# Create Prompt
response = bedrock_agent.create_prompt(
    name=f"BedrockKnowledgeBaseAssistant-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="Prompt for Bedrock Knowledge Base Assistant",
    variants=[
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 3000,
                    "temperature": 0,
                    "topP": 0.1,
                    "topK": 250,
                }
            },
            "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
            "name": "variant-001",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "user_question"},
                        {"name": "knowledge_base_results"}
                    ],
                    "text": """You are an AI as

### Execution

After updating the generated code with your inputs on the required configurations, you can simple execute the generated code that is saved as a python file.

Please review the generated code before you execute it and see if everything is correct. 

If there was an error(s) in the code, the execution module will tell you what went wrong with the code.

In [ ]:
import boto3
from datetime import datetime
import json
import sagemaker

role = sagemaker.get_execution_role()

bedrock_agent = boto3.client(service_name="bedrock-agent", region_name="us-west-2")

# Create the prompt
response = bedrock_agent.create_prompt(
    name=f"AIAssistantPrompt-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="AI Assistant prompt for Amazon Bedrock Knowledge Base",
    variants=[
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 3000,
                    "temperature": 0,
                    "topP": 0.1,
                    # "topK": 250,
                }
            },
            "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
            "name": "variant-001",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "user_query"},
                        {"name": "kb_results"}
                    ],
                    "text": """You are an AI assistant who is well-versed in Amazon Bedrock Knowledge base designed to answer a user's question.
Please generate a code about Amazon Bedrock Knowledge Base only when a user asks for it.
Do NOT use your own knowledge as facts in answers. 
Please output a list in order of the most valuable data sources for a knowledge base when appropriate.
Don't restate the instructions.

User query: {{user_query}}

Knowledge base results: {{kb_results}}

Please provide a response based on the above information:"""
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant="variant-001"
)

prompt_arn = response["arn"]
print(f"Prompt ARN: {prompt_arn}")

# Create the flow
flow_role = role #"arn:aws:iam::123456789012:role/BedrockFlowRole"  # Replace with your actual role ARN

response = bedrock_agent.create_flow(
    name=f"BedrockKnowledgeBaseFlow-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="Flow for answering questions about Amazon Bedrock Knowledge Base",
    executionRoleArn=flow_role,
    definition={
        "nodes": [
            {
                "name": "UserInput",
                "type": "Input",
                "configuration": {
                    "input": {}
                },
                "outputs": [
                    {
                        "name": "document",
                        "type": "String"
                    }
                ],
            },
            {
                "name": "BedrockKnowledgeBase",
                "type": "KnowledgeBase",
                "configuration": {
                    "knowledgeBase": {
                        "knowledgeBaseId": "1CQUXYB3VM"  # Replace with your actual knowledge base ID
                    }
                },
                "inputs": [
                    {
                        "name": "retrievalQuery",
                        "type": "String",
                        "expression": "$.data"
                    }
                ],
                "outputs": [
                    {
                        "name": "retrievalResults",
                        "type": "Array"
                    }
                ],
            },
            {
                "name": "AIAssistant",
                "type": "Prompt",
                "configuration": {
                    "prompt": {
                        "sourceConfiguration": {
                            "resource": {
                                "promptArn": prompt_arn
                            }
                        }
                    }
                },
                "inputs": [
                    {
                        "name": "user_query",
                        "type": "String",
                        "expression": "$.data"
                    },
                    {
                        "name": "kb_results",
                        "type": "Array",
                        "expression": "$.data"
                    }
                ],
                "outputs": [
                    {
                        "name": "modelCompletion",
                        "type": "String"
                    }
                ],
            },
            {
                "name": "FinalResponse",
                "type": "Output",
                "configuration": {
                    "output": {}
                },
                "inputs": [
                    {
                        "name": "document",
                        "type": "String",
                        "expression": "$.data"
                    }
                ],
            }
        ],
        "connections": [
            {
                "name": "Connection_1",
                "source": "UserInput",
                "target": "BedrockKnowledgeBase",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "document",
                        "targetInput": "retrievalQuery"
                    }
                }
            },
            {
                "name": "Connection_2",
                "source": "UserInput",
                "target": "AIAssistant",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "document",
                        "targetInput": "user_query"
                    }
                }
            },
            {
                "name": "Connection_3",
                "source": "BedrockKnowledgeBase",
                "target": "AIAssistant",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "retrievalResults",
                        "targetInput": "kb_results"
                    }
                }
            },
            {
                "name": "Connection_4",
                "source": "AIAssistant",
                "target": "FinalResponse",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "modelCompletion",
                        "targetInput": "document"
                    }
                }
            }
        ],
    }
)

flow_id = response["id"]
flow_arn = response["arn"]
flow_name = response["name"]
print(f"Flow ID: {flow_id}")
print(f"Flow ARN: {flow_arn}")
print(f"Flow Name: {flow_name}")

# Prepare the flow
response = bedrock_agent.prepare_flow(flowIdentifier=flow_id)
print(json.dumps(response, indent=2, default=str))

In [6]:
output = pf_code_execution( updated_output_file_path )

Success: CompletedProcess(args=['python', 'prompt_flow_test/prompt_flow_aws_rag_user_input.py'], returncode=0, stdout='Prompt ARN: arn:aws:bedrock:us-east-1:613305653031:prompt/LDBRVL9N71\nFlow ID: YDBDMXXPNB\nFlow ARN: arn:aws:bedrock:us-east-1:613305653031:flow/YDBDMXXPNB\nFlow Name: BedrockKnowledgeBaseFlow-20241104-165007\n{\n  "ResponseMetadata": {\n    "RequestId": "28025489-93d8-47a6-898b-a3e7e718aaff",\n    "HTTPStatusCode": 202,\n    "HTTPHeaders": {\n      "date": "Tue, 05 Nov 2024 00:50:08 GMT",\n      "content-type": "application/json",\n      "content-length": "40",\n      "connection": "keep-alive",\n      "x-amzn-requestid": "28025489-93d8-47a6-898b-a3e7e718aaff",\n      "x-amz-apigw-id": "Av3GJF1SoAMEZOA=",\n      "x-amzn-trace-id": "Root=1-67296bc0-6470c99433f4271c1f7512ed"\n    },\n    "RetryAttempts": 0\n  },\n  "id": "YDBDMXXPNB",\n  "status": "Preparing"\n}\n', stderr='')


### Check the generated Prompt Flow 

After successfully executing the prompt flow deployment code, your prompt flow is deployed to your AWS account. 

Please go to your AWS console and go to Amazon Bedrock.

In the left panel, go to Prompt flows (preview) under Builder's tool.

Your prompt flow will be in the prompt flows list in the main panel. 

![alt text](prompt_flow_asset/example_pf_list.png "Prompt Flows List")


Click the generate prompt flow and Click Edit in prompt flow builder button in the top right corner.

![alt text](prompt_flow_asset/example_pf_panel.png "Prompt Flow Panel")

You will be able to see the draft of the generated prompt flow and test it with your questions about Amazon Bedrock Knowledge Base.

![alt text](prompt_flow_asset/example_pf_output.png "Automatically Generated Prompt Flow")

Thank you for following this example all the way to the end. Have fun building!